# Monitor and Manage Google Cloud Resources

## Task 1. Create a bucket

In [ ]:
# Activate Cloud Shell

gcloud auth list

gcloud config list project

gcloud config set compute/region "REGION"

gcloud services enable cloudfunctions.googleapis.com

In [ ]:
# Create a bucket

gsutil mb gs://<YOUR-BUCKET-NAME>

1. Select **Navigation menu > IAM & Admin > IAM**.

2. Click **+GRANT ACCESS** button and paste the *User 2* name into the **New principals** field.

3. In the **Select a role** field, select **Cloud Storage > Storage Object Viewer** from the drop-down menu. Click **SAVE**.

## Task 2. Create a Pub/Sub topic

In [ ]:
gcloud pubsub topics create "Topic"

gcloud pubsub topics list

## Task 3. Create the thumbnail Cloud Function

In [ ]:
mkdir gcf_thumbnail

cd gcf_thumbnail

In [ ]:
cat > index.js <<EOF
/* globals exports, require */
//jshint strict: false
//jshint esversion: 6
"use strict";
const crc32 = require("fast-crc32c");
const { Storage } = require('@google-cloud/storage');
const gcs = new Storage();
const { PubSub } = require('@google-cloud/pubsub');
const imagemagick = require("imagemagick-stream");

exports.thumbnail = (event, context) => {
  const fileName = event.name;
  const bucketName = event.bucket;
  const size = "64x64"
  const bucket = gcs.bucket(bucketName);
  const topicName = "REPLACE_WITH_YOUR_TOPIC ID";
  const pubsub = new PubSub();
  if ( fileName.search("64x64_thumbnail") == -1 ){
    // doesn't have a thumbnail, get the filename extension
    var filename_split = fileName.split('.');
    var filename_ext = filename_split[filename_split.length - 1];
    var filename_without_ext = fileName.substring(0, fileName.length - filename_ext.length );
    if (filename_ext.toLowerCase() == 'png' || filename_ext.toLowerCase() == 'jpg'){
      // only support png and jpg at this point
      console.log(`Processing Original: gs://${bucketName}/${fileName}`);
      const gcsObject = bucket.file(fileName);
      let newFilename = filename_without_ext + size + '_thumbnail.' + filename_ext;
      let gcsNewObject = bucket.file(newFilename);
      let srcStream = gcsObject.createReadStream();
      let dstStream = gcsNewObject.createWriteStream();
      let resize = imagemagick().resize(size).quality(90);
      srcStream.pipe(resize).pipe(dstStream);
      return new Promise((resolve, reject) => {
        dstStream
          .on("error", (err) => {
            console.log(`Error: ${err}`);
            reject(err);
          })
          .on("finish", () => {
            console.log(`Success: ${fileName} → ${newFilename}`);
              // set the content-type
              gcsNewObject.setMetadata(
              {
                contentType: 'image/'+ filename_ext.toLowerCase()
              }, function(err, apiResponse) {});
              pubsub
                .topic(topicName)
                .publisher()
                .publish(Buffer.from(newFilename))
                .then(messageId => {
                  console.log(`Message ${messageId} published.`);
                })
                .catch(err => {
                  console.error('ERROR:', err);
                });
          });
      });
    }
    else {
      console.log(`gs://${bucketName}/${fileName} is not an image I can handle`);
    }
  }
  else {
    console.log(`gs://${bucketName}/${fileName} already has a thumbnail`);
  }
};
EOF

In [ ]:
cat > package.json <<EOF
{
  "name": "thumbnails",
  "version": "1.0.0",
  "description": "Create Thumbnail of uploaded image",
  "scripts": {
    "start": "node index.js"
  },
  "dependencies": {
    "@google-cloud/pubsub": "^2.0.0",
    "@google-cloud/storage": "^5.0.0",
    "fast-crc32c": "1.0.4",
    "imagemagick-stream": "4.1.1"
  },
  "devDependencies": {},
  "engines": {
    "node": ">=4.3.2"
  }
}
EOF

In [ ]:
gcloud functions deploy Cloud Function Name \
    --stage-bucket=gs://<YOUR-BUCKET-NAME> \
    --trigger-topic="Topic" \
    --runtime nodejs14 \
    --entry-point="thumbnail" \
    

## Task 4. Create an alerting policy

1. In the left menu, click **Alerting**, and then click **+Create Policy**.

2. Click on **Select a metric** dropdown. Uncheck the **Active**.

3. Select **Cloud Function > Function > Active Instances** and click **Apply**. Click **Next**.

4. Set the **Threshold position** to **Above threshold**, **Threshold value** to **1**. Click **Next**.

5. Click on the drop down arrow next to **Notification Channels**, then click on **Manage Notification Channels**.

6. Scroll down the page and click on **ADD NEW for Email**.

7. In the **Create Email Channel** dialog box, enter your personal email address in the **Email Address** field and a **Display name**. Click **Save**.

8. Click on **Notification Channels** again, then click on the **Refresh** icon, select the **Display name** and click **OK**.

9. Add a message "The number of active Cloud Function instances is greater than zero!" in documentation.

10. Mention the **Alert name** as **Active Cloud Function Instances**. Click **Next**.

11. Click **Create Policy**.